<a href="https://colab.research.google.com/github/ttekcor/neurontasksfefu/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow.keras

ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import files
import opendatasets as od
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import gc


In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
od.download(
    "https://www.kaggle.com/datasets/gpiosenka/100-bird-species")

100%|██████████| 1.96G/1.96G [00:16<00:00, 125MB/s]


In [ ]:

! unzip 100-bird-species -d train

In [9]:
def load_and_preprocess_image(filepath, label):
    # Read and decode the image
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=3)

    # Set the shape of the image tensor explicitly
    img.set_shape([None, None, 3])

    # Resize and normalize the image
    img = tf.image.resize(img, (64, 64))
    img = img / 255.0

    return img, label

def load_dataset(path: str, batch_size: int, image_size: tuple[int, int], shuffle: bool, split: str) -> tuple[tf.data.Dataset, dict[int, str]]:
    # Load the CSV index file
    df = pd.read_csv(path)

    # Filter the dataframe based on the specified split
    df_split = df[df['data set'] == split]

    # Get the filepaths and labels
    filepaths = df_split['filepaths'].values
    labels = df_split['labels'].values

    # Create a dictionary mapping class indices to class names
    class_mapping = {idx: label for idx, label in enumerate(df['labels'].unique())}

    # Create a list of class indices corresponding to the labels
    class_indices = [list(class_mapping.keys())[list(class_mapping.values()).index(label)] for label in labels]

    # Create a dataset from the filepaths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, class_indices))

    # Function to load and preprocess images
    def load_and_preprocess_image1(filepath, label):
        return load_and_preprocess_image(filepath, label)


    # Map the load_and_preprocess_image function to the dataset
    dataset = dataset.map(load_and_preprocess_image1)

    # Shuffle the dataset if required
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(filepaths))

    # Batch and prefetch the dataset
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset, class_mapping



NameError: name 'img' is not defined

In [12]:

# Assuming load_dataset is the function you defined earlier

dataset, class_mapping = load_dataset('/content/100-bird-species/birds.csv', batch_size=32, image_size=(64, 64), shuffle=True, split='train')

# Convert dataset to numpy arrays
dataset_size = len(list(dataset))

# Calculate the sizes of training and validation sets
train_size = int(0.8 * dataset_size)
validation_size = dataset_size - train_size

# Split the dataset into training and validation sets
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)
# Convert back to TensorFlow datasets
train_dataset = train_dataset.map(lambda x,y: load_and_preprocess_image('/content/100-bird-species/birds.csv',train_dataset))
validation_dataset = validation_dataset.map(lambda x,y: load_and_preprocess_image('/content/100-bird-species/birds.csv',validation_dataset))



In [24]:


def create_model(input_shape=(64, 64, 3), num_classes=2):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output before feeding into dense layers
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Optional dropout for regularization

    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Create the model
model = create_model(input_shape=(64, 64, 3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Display model summary
model.summary()

# Example usage (replace this with your actual data loading):
# train_dataset, _ = load_dataset('/path/to/train.csv', batch_size=32, image_size=(64, 64), shuffle=True, split='train')
# validation_dataset, _ = load_dataset('/path/to/validation.csv', batch_size=32, image_size=(64, 64), shuffle=False, split='valid')

# Assuming you have a dataset without the batch dimension (64, 64, 3)
# Add a batch dimension to your datasets
# Assuming you have a dataset without the batch dimension (64, 64, 3)
# Add a batch dimension to your datasets
train_dataset = train_dataset.map(lambda x, y: (tf.expand_dims(x, axis=0), y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.expand_dims(x, axis=0), y))

# Now, check the shape of your dataset samples
for sample in train_dataset.take(1):
    print("Input Shape:", sample[0].shape)
    print("Label Shape:", sample[1].shape)



Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_28 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 6, 6, 128)       

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

In [20]:
train_dataset


<_MapDataset element_spec=(TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None), DatasetSpec((TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None)), TensorShape([])))>

In [21]:
validation_dataset

<_MapDataset element_spec=(TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None), DatasetSpec((TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None)), TensorShape([])))>